### 1. **BFMatcher (Brute-Force Matcher)**
   - Compares descriptors between keypoints in a brute-force manner.
   - Supports different norms for distance calculation, such as `cv::NORM_L2` or `cv::NORM_HAMMING`.
   - Suitable for small datasets.
   - Example:
     ```cpp
     cv::BFMatcher matcher(cv::NORM_L2);
     std::vector<cv::DMatch> matches;
     matcher.match(descriptors1, descriptors2, matches);
     ```


#### Distance Types
- `L2 Norm` : `cv.NORM_L2` Euclidean distance, used mainly for floating-point descriptors like SIFT and SURF.
By default, it is `cv.NORM_L2`. It is good for SIFT, SURF etc (`cv.NORM_L1 is` also there). 

```
bf = cv2.BFMatcher(cv2.cv.NORM_L2, crossCheck=True)
```


- `Hamming Distance`: `cv.NORM_HAMMING` Used for binary string-based descriptors like `ORB, BRIEF, and BRISK`. It counts the number of differing bits between two binary strings. If ORB is using `WTA_K == 3` or `4`, which takes 3 or 4 points to produce BRIEF descriptor, `cv.NORM_HAMMING2` should be used.


```
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
```

- `Cross Check`: An option in the BFMatcher that ensures mutual matching. For two keypoints to be considered a match, the keypoint in the first image must match the keypoint in the second image, and vice-versa.


### 2. **FlannBasedMatcher (Fast Library for Approximate Nearest Neighbors)**
   - Efficient for large datasets by using an approximate nearest neighbor algorithm.
   - Suitable for high-dimensional data and faster than BFMatcher for larger datasets.
   - Example:
     ```cpp
     cv::FlannBasedMatcher matcher;
     std::vector<cv::DMatch> matches;
     matcher.match(descriptors1, descriptors2, matches);
     ```

**Additional Matching Techniques**
   OpenCV also allows for custom matchers or modifications:
   
   - **Radius Match**: Finds all descriptors within a specified distance.
     ```cpp
     matcher.radiusMatch(descriptors1, descriptors2, knnMatches, maxDistance);
     ```

   - **K-Nearest Neighbors (KNN) Match**: Finds the k best matches for each descriptor.
     ```cpp
     matcher.knnMatch(descriptors1, descriptors2, knnMatches, k);
     ```

   - **Cross-Check Matching**: A refinement where matches are validated bidirectionally.

### Summary of Norm Types for Descriptors
   The choice of matcher depends on the type of descriptor used:
   - **SIFT/SURF**: Use `cv::NORM_L2` with BFMatcher or FlannBasedMatcher.
   - **ORB/BRIEF/BRISK**: Use `cv::NORM_HAMMING` or `cv::NORM_HAMMING2` with BFMatcher.

### Example Using  Matchers
Here’s a combined example with `BFMatcher` and `FlannBasedMatcher`:
```cpp
#include <opencv2/opencv.hpp>
#include <opencv2/features2d.hpp>

int main() {
    cv::Mat img1 = cv::imread("image1.jpg", cv::IMREAD_GRAYSCALE);
    cv::Mat img2 = cv::imread("image2.jpg", cv::IMREAD_GRAYSCALE);

    cv::Ptr<cv::Feature2D> detector = cv::ORB::create();
    std::vector<cv::KeyPoint> keypoints1, keypoints2;
    cv::Mat descriptors1, descriptors2;

    detector->detectAndCompute(img1, cv::noArray(), keypoints1, descriptors1);
    detector->detectAndCompute(img2, cv::noArray(), keypoints2, descriptors2);

    // BFMatcher
    cv::BFMatcher bfMatcher(cv::NORM_HAMMING);
    std::vector<cv::DMatch> bfMatches;
    bfMatcher.match(descriptors1, descriptors2, bfMatches);

    // FlannBasedMatcher
    cv::FlannBasedMatcher flannMatcher;
    std::vector<cv::DMatch> flannMatches;
    flannMatcher.match(descriptors1, descriptors2, flannMatches);

    return 0;
}
```

### Full Example with `knnMatch`

```cpp
#include <opencv2/opencv.hpp>
#include <opencv2/features2d.hpp>
#include <iostream>

int main() {
    // Read input images
    cv::Mat img1 = cv::imread("image1.jpg", cv::IMREAD_GRAYSCALE);
    cv::Mat img2 = cv::imread("image2.jpg", cv::IMREAD_GRAYSCALE);

    if (img1.empty() || img2.empty()) {
        std::cerr << "Could not load images!" << std::endl;
        return -1;
    }

    // Detect ORB keypoints and compute descriptors
    cv::Ptr<cv::ORB> detector = cv::ORB::create();
    std::vector<cv::KeyPoint> keypoints1, keypoints2;
    cv::Mat descriptors1, descriptors2;

    detector->detectAndCompute(img1, cv::noArray(), keypoints1, descriptors1);
    detector->detectAndCompute(img2, cv::noArray(), keypoints2, descriptors2);

    if (descriptors1.empty() || descriptors2.empty()) {
        std::cerr << "Descriptors could not be computed!" << std::endl;
        return -1;
    }

    // Use BFMatcher with NORM_HAMMING (suitable for ORB descriptors)
    cv::BFMatcher matcher(cv::NORM_HAMMING, /*crossCheck=*/false);

    // Perform KNN matching (k=2)
    std::vector<std::vector<cv::DMatch>> knnMatches;
    matcher.knnMatch(descriptors1, descriptors2, knnMatches, 2);

    // Apply Lowe's ratio test to filter matches
    const float ratioThresh = 0.75f; // Lowe's ratio test threshold
    std::vector<cv::DMatch> goodMatches;
    for (const auto& knnMatch : knnMatches) {
        if (knnMatch.size() >= 2 && knnMatch[0].distance < ratioThresh * knnMatch[1].distance) {
            goodMatches.push_back(knnMatch[0]);
        }
    }

    // Draw matches
    cv::Mat imgMatches;
    cv::drawMatches(img1, keypoints1, img2, keypoints2, goodMatches, imgMatches,
                    cv::Scalar::all(-1), cv::Scalar::all(-1), std::vector<char>(),
                    cv::DrawMatchesFlags::NOT_DRAW_SINGLE_POINTS);

    // Display the result
    cv::imshow("Good Matches", imgMatches);
    cv::waitKey(0);

    return 0;
}
```

### Explanation of Key Steps:
1. **Feature Detection and Description**:
   - `cv::ORB::create()` detects ORB keypoints and computes descriptors.

2. **Matcher Selection**:
   - `cv::BFMatcher` with `cv::NORM_HAMMING` is used, as ORB descriptors are binary.

3. **KNN Matching**:
   - `matcher.knnMatch()` finds the two best matches for each descriptor.

4. **Lowe's Ratio Test**:
   - Compares the distances of the two best matches to filter out ambiguous matches.

5. **Visualization**:
   - `cv::drawMatches()` visualizes the filtered matches.

### Customization:
- Replace `cv::ORB` with another feature detector like `cv::SIFT` or `cv::AKAZE` as needed.
- Adjust `ratioThresh` for stricter or more lenient filtering.

### Example of `radiusMatch` (Complete Code):

```cpp
#include <opencv2/opencv.hpp>
#include <opencv2/features2d.hpp>
#include <iostream>

int main() {
    // Read input images
    cv::Mat img1 = cv::imread("image1.jpg", cv::IMREAD_GRAYSCALE);
    cv::Mat img2 = cv::imread("image2.jpg", cv::IMREAD_GRAYSCALE);

    if (img1.empty() || img2.empty()) {
        std::cerr << "Could not load images!" << std::endl;
        return -1;
    }

    // Detect ORB keypoints and compute descriptors
    cv::Ptr<cv::ORB> detector = cv::ORB::create();
    std::vector<cv::KeyPoint> keypoints1, keypoints2;
    cv::Mat descriptors1, descriptors2;

    detector->detectAndCompute(img1, cv::noArray(), keypoints1, descriptors1);
    detector->detectAndCompute(img2, cv::noArray(), keypoints2, descriptors2);

    if (descriptors1.empty() || descriptors2.empty()) {
        std::cerr << "Descriptors could not be computed!" << std::endl;
        return -1;
    }

    // Use BFMatcher with NORM_HAMMING (suitable for ORB descriptors)
    cv::BFMatcher matcher(cv::NORM_HAMMING);

    // Perform radius matching
    const float maxDistance = 50.0f; // Radius threshold
    std::vector<std::vector<cv::DMatch>> radiusMatches;
    matcher.radiusMatch(descriptors1, descriptors2, radiusMatches, maxDistance);

    // Filter and collect matches for visualization
    std::vector<cv::DMatch> goodMatches;
    for (const auto& matches : radiusMatches) {
        for (const auto& match : matches) {
            if (match.distance < maxDistance) {
                goodMatches.push_back(match);
            }
        }
    }

    // Draw matches
    cv::Mat imgMatches;
    cv::drawMatches(img1, keypoints1, img2, keypoints2, goodMatches, imgMatches);

    // Display the result
    cv::imshow("Good Matches (Radius Match)", imgMatches);
    cv::waitKey(0);

    return 0;
}
```

### When to Use Which:
- Use `knnMatch` when you need a fixed number of best matches for each descriptor (e.g., for applying Lowe's ratio test).
- Use `radiusMatch` when you want to consider all matches within a spatial or distance threshold, especially for applications where proximity is more critical than ranking.

The **distance** in OpenCV's matchers (such as `knnMatch` or `radiusMatch`) is not measured in pixels. Instead, it represents the **difference between feature descriptors**. The nature of this distance depends on:

### 1. **Descriptor Type**
   - The type of feature descriptor (e.g., ORB, SIFT, SURF) determines how the distance is calculated.

### 2. **Norm Type (Distance Metric)**
   - When creating a matcher (e.g., `cv::BFMatcher`), you specify a **norm type**, which defines the mathematical metric used to compute the distance. Common norms include:
     - **`cv::NORM_L2` (Euclidean Distance):**
       - Used for floating-point descriptors like SIFT and SURF.
       - Distance = √(Σ(descriptor1[i] - descriptor2[i])²)
     - **`cv::NORM_HAMMING` (Hamming Distance):**
       - Used for binary descriptors like ORB, BRIEF, and BRISK.
       - Distance = Number of differing bits between two binary descriptors.

### 3. **Physical Meaning**
   - The "distance" is purely a measure of similarity between feature descriptors:
     - Smaller distance = Higher similarity.
     - Larger distance = Lower similarity.
   - It does not directly correspond to spatial distances in pixels.

### Example Interpretation:
- **ORB Descriptor with `cv::NORM_HAMMING`:**
  - A distance of `5` means there are 5 differing bits between the binary descriptors.
- **SIFT Descriptor with `cv::NORM_L2`:**
  - A distance of `10.5` means the Euclidean distance between two descriptors in the feature space is `10.5`.

### Choosing Distance Thresholds:
- When filtering matches (e.g., in `radiusMatch` or Lowe's ratio test), the distance threshold depends on the descriptor type:
  - **ORB (Binary Descriptors):** A typical `maxDistance` is in the range of `30–50` (Hamming distance).
  - **SIFT/SURF (Floating-Point Descriptors):** A typical `maxDistance` might be `0.5–1.0` (L2 norm).

---

### Example:
If you’re using ORB and set `maxDistance = 50` in `radiusMatch`:
- The matcher will consider all descriptors from the second image that are within 50 bits difference (Hamming distance) of a descriptor from the first image.

If you’re using SIFT and set `maxDistance = 1.0`:
- The matcher will consider all descriptors within a Euclidean distance of 1.0.

### Summary:
The "distance" reflects the difference between descriptors in their respective feature spaces, **not physical pixel distances in the image**. It is a similarity measure for matching purposes.

Lowe's ratio test is a widely-used method to filter out ambiguous or incorrect matches when performing feature matching. Here's a detailed explanation of what it is, how it works, and why we compare only `knnMatch[0].distance` with `ratioThresh * knnMatch[1].distance`.

---

### What is Lowe's Ratio Test?

- It is a **validation step** that helps distinguish between good matches (true correspondences) and poor or ambiguous matches (false correspondences).
- The idea is based on the assumption that a good match should have a **significantly better similarity score** (smaller distance) than the next best alternative match.

---

### How Does it Work?

1. **`knnMatch` Results**:
   - For each descriptor in the first image, the `knnMatch` function retrieves the top `k` closest matches from the descriptors in the second image.
   - Typically, `k=2` is used to retrieve the two best matches: `knnMatch[0]` (best match) and `knnMatch[1]` (second-best match).

2. **Ratio Test**:
   - Compare the distance of the best match (`knnMatch[0].distance`) to the distance of the second-best match (`knnMatch[1].distance`).
   - If the ratio of these distances is below a predefined threshold (e.g., 0.75), it indicates a strong match. Otherwise, the match is considered ambiguous and discarded.

   Mathematically:
   $
   \text{If } \frac{\text{knnMatch[0].distance}}{\text{knnMatch[1].distance}} < \text{ratioThresh}, \text{ accept the match.}
   $

   - **Typical `ratioThresh` Values**:
     - The default threshold is often **0.7** or **0.75** (Lowe's original paper recommends 0.75).

---

### Why Compare Only the Two Best Matches?

The comparison between the best match (`knnMatch[0]`) and the second-best match (`knnMatch[1]`) provides a measure of confidence in the match:

1. **Ambiguity Detection**:
   - If the best match (`knnMatch[0]`) is very similar in distance to the second-best match (`knnMatch[1]`), the descriptor is likely ambiguous.
   - Ambiguity often occurs when two or more features in the second image are very similar to a feature in the first image.

2. **Robustness to Noise**:
   - By considering only the top two matches, the test avoids errors introduced by noisy or irrelevant descriptors.

3. **Efficiency**:
   - Examining just the top two matches reduces computational complexity while still providing robust filtering.

---

### Example of Lowe's Ratio Test in Practice:

```cpp
const float ratioThresh = 0.75f; // Lowe's ratio threshold
std::vector<cv::DMatch> goodMatches;

for (const auto& knnMatch : knnMatches) {
    if (knnMatch.size() >= 2) { // Ensure at least two matches exist
        // Apply Lowe's ratio test
        if (knnMatch[0].distance < ratioThresh * knnMatch[1].distance) {
            goodMatches.push_back(knnMatch[0]);
        }
    }
}
```

---

### Visualization:

#### Example:
Imagine the distances for a descriptor in image 1 are:
- `knnMatch[0].distance = 0.3` (best match)
- `knnMatch[1].distance = 0.5` (second-best match)

$
\text{Ratio: } \frac{0.3}{0.5} = 0.6
$

If `ratioThresh = 0.75`, this match passes the test.

#### Counterexample:
Now imagine the distances are:
- `knnMatch[0].distance = 0.4`
- `knnMatch[1].distance = 0.42`

$
\text{Ratio: } \frac{0.4}{0.42} \approx 0.95
$

This match fails the test because the best match is not significantly better than the second-best match, indicating ambiguity.

---

### Why Not Compare More Matches?

- Comparing more than two matches would increase computational overhead without significantly improving robustness.
- Lowe's ratio test is designed to identify **clear, unambiguous matches**. If the first two matches are ambiguous, adding more matches is unlikely to resolve the ambiguity.

---

### Summary:
- Lowe's ratio test ensures that the best match is **distinctively better** than the second-best match, reducing false positives.
- It relies only on `knnMatch[0]` and `knnMatch[1]` for simplicity and efficiency.
- The ratio threshold (e.g., `0.75`) is an empirically determined value that balances filtering out ambiguous matches and retaining valid ones.

[c++ code](../src/correspondences_matching.cpp)